In [6]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


df = pd.read_csv('processed.csv', index_col=0)

X = df.drop(labels=['Price'], axis=1, inplace=False)
y = df['Price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
lin_reg = LinearRegression()

result = lin_reg.fit(X_train, y_train)


In [ ]:
result.score(X_test, y_test)